In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import statsmodels.api as sm
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
from matplotlib import cm
import os

In [3]:
df_iv = pd.read_spss('data/caspian4_.sav',convert_categoricals=True)
df_v = pd.read_spss('data/last-caspian-v.sav',convert_categoricals=True) 
def rename_features(df1, caspian_number):
    df2 = df1.copy()
    # Rename the features in the dataframe
    if caspian_number == 5:
        df2.rename(columns={'weight_1': 'weight', 'height_2': 'height', 'universi': 'university','ap_9':'schoolType'}, inplace=True)
    elif caspian_number == 4:
        df2.drop('sex', axis=1, inplace=True)
        print(df2.columns)
        df2.rename(columns={'sex2': 'sex', 'weight_1': 'weight', 'height_2': 'height', 'University': 'university','ap_9':'schoolType'}, inplace=True)
    elif caspian_number == 3:
        df2.rename(columns={'weighte': 'weight', 'heighte': 'height', 'province': 'university', 'area':'region','p9':'schoolType'}, inplace=True)
    elif caspian_number == 1:
        df2.columns = df.columns.str.lower()
        df2.rename(columns={'univer': 'university','district':'region', 'schoolty':'schoolType'}, inplace=True)
        

    return df2

df_iv=rename_features(df_iv,4)

df_v = rename_features(df_v,5)
df_iv['sex'] = df_iv['sex'].apply(lambda x: 'Girl' if x == 'girl' else 'Boy' if x == 'boy' else x)
df_iv['schoolType']=df_iv['schoolType'].apply(lambda x: 'Public School' if x=='dolati' else 'Private School' if x=='gheyre entef' else 'Unknown') 
df_v['schoolType']=df_v['schoolType'].apply(lambda x: 'Public School' if x=='dolati' else 'Private School' if x=='gheyre entefai' else 'Unknown') 
df_dict = {'caspian_IV': df_iv, 'caspian_V': df_v}


def preprocess(dataframes_dict):
    processed_dfs = {}  # Dictionary to store processed DataFrames
    for name, df_org in dataframes_dict.items():
        df = df_org.copy()

        # Filter age
        df = df[(df["age"] >= 7) & (df["age"] <= 18)]
        # please change the type of heught_1 and weight_1 in caspian4 to numeric if you can't do it directly uncomment two line below
        df['height'] = pd.to_numeric(df['height'], errors='coerce')
        df['weight'] = pd.to_numeric(df['weight'], errors='coerce')
        df["bmi1"] = df["weight"] / ((df["height"] / 100) ** 2)
        
        # Remove null tuples
        records_with_nulls = df[
            df[["weight", "height", "sex", "age"]].isna().any(axis=1)
        ]
        df = df.dropna(subset=["height", "weight", "sex"])
        print(
            f"Number of records with NaN value in weight or height in {name}: {len(records_with_nulls)}"
        )

        # Store the processed DataFrame in the new dictionary
        processed_dfs[name] = df

    return processed_dfs


# Usage
processed_dfs = preprocess(df_dict)

# Define the mapping dictionary
replacement_map = {
    '1 or 2 in we': '1 or 2 in week',
    '3 or 4 in we': '3 or 4 in week',
    '5 or 6 in we': '5 or 6 in week',
    'never': 'never',
    'everyday': 'everyday',
    '': np.nan  # map empty strings to NaN
}
replacement_ap_3= {
    'diplome': 'diplome',
    'bachelor': 'Bachelor',
    'primary': 'primary',
    'upper than b': 'upper than bachelor',
    '': np.nan,  # Empty string replaced by NaN
    'intermediate': 'intermediate',
    'illiterate': 'illiterate',
    'father died': 'Father died',
    'quranic lite': 'Quranic Literacy',
    'mother died' :'mother died'
}


# Apply the mapping to 'a_4' column, using the default value of an empty string
processed_dfs['caspian_IV']['a_4'] = processed_dfs['caspian_IV']['a_4'].apply(lambda x: replacement_map.get(x, ''))
processed_dfs['caspian_IV']['ap_3'] = processed_dfs['caspian_IV']['ap_3'].apply(lambda x: replacement_ap_3.get(x, ''))
processed_dfs['caspian_IV']['ap_4'] = processed_dfs['caspian_IV']['ap_4'].apply(lambda x: replacement_ap_3.get(x, ''))


Index(['id2', 'University', 'region', 'cluster', 'cross', 'birth_ye',
       'sample_c', 'a_1', 'a_2', 'a_3',
       ...
       'HeiMeter', 'bmi', 'CATBMI', 'CATBMIoutlieer', 'dicBMI',
       'socialsupport', 'PA1', 'PA2', 'KVDM', 'obes'],
      dtype='object', length=410)
Number of records with NaN value in weight or height in caspian_IV: 143
Number of records with NaN value in weight or height in caspian_V: 128


In [10]:
def compare_feature_names(df1, df2):
    # Get sets of column names for each DataFrame
    columns_df1 = set(df1.columns)
    columns_df2 = set(df2.columns)

    # Find columns that are only in df1 and only in df2
    unique_to_df1 = columns_df1 - columns_df2
    unique_to_df2 = columns_df2 - columns_df1

    # Display results
    if unique_to_df1:
        print(f"Columns only in caspian_IV dataset: {unique_to_df1}")
    else:
        print("No unique columns in the caspian_IV dataset.")

    if unique_to_df2:
        print(f"Columns only in caspian_V dataset: {unique_to_df2}")
    else:
        print("No unique columns in the caspian_V dataset.")
    return unique_to_df1,unique_to_df2

# Usage
unique_to_df1,unique_to_df2=compare_feature_names(processed_dfs['caspian_IV'], processed_dfs['caspian_V'])


Columns only in caspian_IV dataset: {'passivewaterpipeother', 'HeiMeter', 'CigPassive', 's_98_0', 'passivewaterpipeMother', 'passivewaterpipefather', 'CatSRH', 'Zoor', 'passivewaterpipeBrotherorsister', 'ZNPW', 'socialsupport', 'PA1', 'obes', 'Vitamin', 'dicBMI', 'WCWL', 'passivecigfather', 'typevitamin', 'KVDM', 'SFR', 'WaterpipePassive', 'SHPD', 'passiveCigother', 'CATBMIoutlieer', 'CATBMI', 'passiveCigBrotherorsister', 'PassiveTotalSmoking1', 'PA', 'PA2', 'dust', 'passiveCigMother'}
Columns only in caspian_V dataset: {'feelingdizzy', 'pacatego', 'homework', 'paday', 'highldlf', 'ldlirani', 'filter_$', 'sleepdur', 'student', 'highldli', 'schoolp0', 'mobilec0', 'excesswe', 'dislipid', 'filter', 'healthyp', 'TYG', 'healthyc', 'highdias', 'dislipem', 'breakfas', 'parentss', 'parent_0', 'Dinnerskipping', 'headache', 'newalt', 'surfing0', 'irritabi', 'bpcompon', 'sex2', 'gfrterti', 'park', 'bi_bw_st', 'parent_o', 'new_comb', 'parentas', 'parent_b', 'zwc', 'Agebinary', 'fatherpa', 'stcateg

In [15]:
for col in unique_to_df1:
    print(f'for column {col} unique values are:',processed_dfs['caspian_IV'][col].unique())

for column passivewaterpipeother unique values are: [ 0.  1. nan]
for column HeiMeter unique values are: [ 2. nan  1.]
for column CigPassive unique values are: [1. 0.]
for column s_98_0 unique values are: [nan]
for column passivewaterpipeMother unique values are: [ 0. nan  1.]
for column passivewaterpipefather unique values are: [ 0.  1. nan]
for column CatSRH unique values are: ['Good or Perfect' 'bad or average!' '']
for column Zoor unique values are: ['>=1' 'Never' '']
for column passivewaterpipeBrotherorsister unique values are: [ 0. nan  1.]
for column ZNPW unique values are: ['' '44652' 'none']
for column socialsupport unique values are: [0. 1.]
for column PA1 unique values are: [ 0.  1. nan]
for column obes unique values are: [4. 3. 2. 1.]
for column Vitamin unique values are: ['no' 'yes' '']
for column dicBMI unique values are: [nan  1.  0.]
for column WCWL unique values are: ['No' 'Yes' '']
for column passivecigfather unique values are: [ 1.  0. nan]
for column typevitamin uni

In [21]:
for col in unique_to_df2:
    vc=processed_dfs['caspian_V'][col].unique()
    print(f'column {col}:',vc)

column feelingdizzy: ['no', NaN, 'yes']
Categories (2, object): ['no', 'yes']
column pacatego: ['moderate', 'low', NaN, 'high']
Categories (3, object): ['high', 'low', 'moderate']
column homework: [2.         3.85714286 1.         1.14285714 4.         2.85714286
 3.         0.85714286        nan 3.14285714 0.14285714 1.85714286
 2.14285714 1.42857143 1.28571429 0.         3.57142857 3.71428571
 2.71428571 2.57142857 2.28571429 1.71428571 3.42857143 0.42857143
 0.28571429 0.57142857]
column paday: ['6-7', '<=2', '3-5', NaN]
Categories (3, object): ['3-5', '6-7', '<=2']
column highldlf: [NaN, 'No', 'Yes']
Categories (2, object): ['No', 'Yes']
column ldlirani: [         nan  73.5640706  147.20485706 ...  83.9741868   94.53709139
  85.36576736]
column filter_$: ['Not Selected', 'Selected']
Categories (2, object): ['Not Selected', 'Selected']
column sleepdur: [ 8.          9.14285714  9.42857143  8.85714286  9.85714286  8.54285731
 10.14285714  8.28571429  9.          7.40000016  7.2857142

In [22]:
def print_unique_values_with_counts(df, unique_columns):
    for col in unique_columns:
        print(f"\nColumn '{col}':")
        
        # Use value_counts with dropna=False to include NaN counts
        value_counts = df[col].value_counts(dropna=False)
        
        # Print each unique value with its count
        for value, count in value_counts.items():
            print(f"  Value: {value} | Count: {count}")

# Usage
unique_to_df2 = set(processed_dfs['caspian_V'].columns) - set(processed_dfs['caspian_IV'].columns)
print_unique_values_with_counts(processed_dfs['caspian_V'], unique_to_df2)



Column 'feelingdizzy':
  Value: no | Count: 12826
  Value: yes | Count: 1113
  Value: nan | Count: 207

Column 'pacatego':
  Value: low | Count: 4424
  Value: high | Count: 4412
  Value: moderate | Count: 4387
  Value: nan | Count: 923

Column 'homework':
  Value: 1.0 | Count: 1945
  Value: 1.1428571428571428 | Count: 1850
  Value: 4.0 | Count: 1783
  Value: 2.0 | Count: 1506
  Value: 2.142857142857143 | Count: 1435
  Value: 3.0 | Count: 1304
  Value: 3.857142857142857 | Count: 778
  Value: 3.142857142857143 | Count: 637
  Value: 2.857142857142857 | Count: 492
  Value: 1.4285714285714286 | Count: 405
  Value: 1.8571428571428572 | Count: 364
  Value: 1.2857142857142858 | Count: 305
  Value: 3.7142857142857144 | Count: 283
  Value: 0.8571428571428571 | Count: 272
  Value: 2.2857142857142856 | Count: 237
  Value: 2.7142857142857144 | Count: 192
  Value: 3.5714285714285716 | Count: 90
  Value: 1.7142857142857142 | Count: 83
  Value: 2.5714285714285716 | Count: 70
  Value: nan | Count: 52
